# LIBRERIAS

In [ ]:
import requests
import pandas as pd
from io import StringIO
from string import ascii_uppercase as alfabeto
import pickle
from bs4 import BeautifulSoup

# DATA EXTRACTION

## EXTRACCION DE MI DATAFRAME OBJETIVO

Importo los grupos desde Wikipedia

In [ ]:
headers = {
    "User-Agent": "PabloWikiBot/1.0 (contacto: pablolube@gmail.com)"
}

url = "https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol_de_2026"

response=requests.get(url,headers=headers)
response.raise_for_status()  # Si falla

#Transformo en texto plano
html=response.text

#Lo cargo en cache en string IO 
df_list=pd.read_html(StringIO(html))

print(f"Cantidad de Tablas {len(df_list)}")




Veo la información que me es relevante para el análisis

In [ ]:
j=0
for i in range(13,97,7):
    j=j+1
    print(f"GRUPO {j}")
    display(df_list[i])

Cambio el nombre, por el nombre real de cada grupo y lo guardo en un diccionario

In [ ]:
alfabeto

In [ ]:
dict_tablas={}
for letra,i in zip(alfabeto,range(13,97,7)):
    df=df_list[i]
    dict_tablas[f"Grupo {letra}"]=df

print(dict_tablas)

In [ ]:
with open("grupos_mundial.pkl", "wb") as f:
    pickle.dump(dict_tablas, f)

In [ ]:
## EXTRACCION DE MI DATAFRAME MUNDIALES HISTORICO 

In [ ]:
import re
def obtener_partidos(año):
    url_all_worlds_cup= "https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol_de_"
    response=requests.get(f"{url_all_worlds_cup}{año}",headers=headers)
    response.raise_for_status()  # Si falla

    #Transformo en texto plano
    content=response.text
    soup=BeautifulSoup(content,'lxml')
    matches = soup.find_all("table", class_="vevent")
    local=[]
    resultado=[]
    visitante=[]
    dict_futbol={}
    for match in matches:
        equipos = match.select("td a[title^='Selección']")

        if len(equipos) < 2:
            continue

        equipo_local = equipos[0].get_text(strip=True)
        equipo_visitante = equipos[1].get_text(strip=True)
        marcador_tag = match.find("td", align="center")
        if not marcador_tag:
            continue

        marcador = marcador_tag.find("b")
        if not marcador:
            continue

        rdo = marcador.get_text(strip=True)
        rdo=re.search(r"\d+:\d+", rdo).group()
        resultado.append(rdo)

        local.append(equipo_local)
        visitante.append(equipo_visitante)

    dict_futbol={'local':local,'resultado':resultado,'visitante':visitante}
    df_futbol=pd.DataFrame(dict_futbol)
    df_futbol["año"]=año
    return df_futbol





In [ ]:
year_world_cup_mundial = [

 1930, 1934, 1938,
    1950, 1954, 1958, 1962, 1966,
    1970, 1974, 1978, 1982,
    1986, 1990, 1994, 1998,
    2002, 2006, 2010, 2014,
    2018, 2022]


headers = {
    "User-Agent": "PabloWikiBot/1.0 (contacto: pablolube@gmail.com)"
}

dfs=[]
for año in year_world_cup_mundial:
    df=obtener_partidos(año)
    dfs.append(df)
df_world_cup_historic = pd.concat(dfs, ignore_index=True)
df_world_cup_historic.to_csv("World_Cup_Results.csv",index=False)



In [ ]:

pd.set_option("display.max_rows", None)
df_world_cup_historic

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get("https://www.google.com")

input("Enter para cerrar")
driver.quit()
